We have a dataset (DCC) with a shortage of certain labels. We want to generate 
new samples synthetically using GPT-4. We will use the following approach:
1. We take the existing samples for each document type and present these to GPT-4
2. we ask to generate new sentences like it, where the token labels are provided in the BIO format

We care specifically about the following labels:
* Experiencer: Other
* Historical: Hypothetical

The task of the GPT model is to generate new sentences that are similar to the input sentences but with variations of the medical concepts. 




# Definitions
The definitions are taken from the ConText/ConTextD papers:

## Negation

This property has two values, ‘Negated’ or ‘Not negated’. A clinical condition or term is labeled as ‘Negated’ if there is evidence in the text suggesting that the condition does not occur or exist, e.g., ‘There was no sign of sinus infection’, otherwise it is ‘Not negated’.

## Temporality

The temporality property places a condition along a time line. There are three possible values for this property: ‘Recent’, ‘Historical’, and ‘Hypothetical’. A condition is considered ‘Recent’ if it is maximally 2 weeks old. Conditions that developed more than 2 weeks ago are labeled as ‘Historical’. A condition is labeled as ‘Hypothetical’ if it is not ‘Recent’ or ‘Historical’, e.g., ‘patient should return if she develops fever’ [13].

**Adaptation**: *'Hypothetical' is specifically about (theoretical) concepts, concepts that are not (yet) realized, i.e. concepts that may materialize in the future. 'Historical' and 'Recent' can be used for realized concepts, in which we also include their negations. I.e. if a concept is explicitly denied historically or recently, we can label it as 'Historical' or 'Recent' respectively.*

## Experiencer

Clinical text may refer to subjects other than the actual patient. The experiencer property describes whether the patient experienced the condition or someone else. For simplicity, we have defined only two possible values for this property: ‘Patient’ or ‘Other’, where ‘Other’ refers to anyone but the actual patient, e.g., ‘Mother is recently diagnosed with cancer’.

In [ ]:
import os, sys, re
import json, dotenv
import pprint

import openai
import asyncio
from openai import AsyncOpenAI, OpenAI
from tqdm import tqdm
from collections import defaultdict

import pandas as pd

import datetime

dotenv.load_dotenv()

In [ ]:
run_temporality = True
run_experiencer = False 
experiencer_file = '../data/synth_experiencer_gpt_4_1106_preview.parquet'
hypothetical_file = '../data/synth_hypothetical_gpt_4_1106_preview.parquet'

In [ ]:
openai.api_key = os.getenv("OPENAI_KEY")

In [ ]:
DCC = json.load(open('../data/emc-dcc_ann.json'))

In [ ]:
update_dcc = DCC.copy()
for c in update_dcc['projects'][0]['documents']:
    c['source'] = 'EMC_DCC_ORIGINAL'
with open('../data/emc-dcc_ann_ORIGNAL.json', 'w') as f:
    json.dump(update_dcc, f, indent=2)

In [ ]:
docs = update_dcc['projects'][0]['documents']

In [ ]:
class_counts = {'Negation': defaultdict(int),
                'Temporality': defaultdict(int),
                'Experiencer': defaultdict(int)}

for doc in docs:
    for ann in doc['annotations']:
        for _class, val in ann['meta_anns'].items():
            class_counts[_class][val['value']] += 1

In [48]:
pprint.pprint(class_counts, indent=2)

{ 'Experiencer': defaultdict(<class 'int'>, {'patient': 12451, 'other': 100}),
  'Negation': defaultdict(<class 'int'>,
                          { 'negated': 1760,
                            'not negated': 10791}),
  'Temporality': defaultdict(<class 'int'>,
                             { 'historical': 511,
                               'hypothetical': 109,
                               'recent': 11931})}


In [ ]:
#len(relevant_docs_hypothetical), len(relevant_docs_experiencer)
#relevant_docs_hypothetical[0]['text'][0:110]
#[(d['start'],d['end'], d['id']) for d in relevant_docs_hypothetical[0]['annotations'] 
#        if d['meta_anns']['Temporality']['value']=='hypothetical']
#relevant_docs_hypothetical[0]['name']

In [ ]:
corrections = [
    {'doc_id': 'DL1616', 'annotation_id': 1873, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'DL1139', 'annotation_id': 108, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'GP2799', 'annotation_id': 8210, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'SP1476', 'annotation_id': 15532, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'DL1567', 'annotation_id': 1694, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL1711', 'annotation_id': 2232, 'meta': 'Temporality', 'value': 'recent'},
    {'doc_id': 'DL1812', 'annotation_id': 2232, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL1814', 'annotation_id': 2703, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'GP1395', 'annotation_id': 4538, 'meta': 'Temporality', 'value': 'recent'},
    {'doc_id': 'DL2111', 'annotation_id': 3779, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL2100', 'annotation_id': 3716, 'meta': 'Temporality', 'value': 'historical'},  
    {'doc_id': 'DL2100', 'annotation_id': 3716, 'meta': 'Temporality', 'value': 'historical'},  
    {'doc_id': 'DL2072', 'annotation_id': 3627, 'meta': 'Temporality', 'value': 'historical'},    
    {'doc_id': 'DL2067', 'annotation_id': 3606, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL1931', 'annotation_id': 3113, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL1812', 'annotation_id': 2689, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL1507', 'annotation_id': 1467, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL1167', 'annotation_id': 212, 'meta': 'Temporality', 'value': 'historical'},
 ]

In [ ]:
# update the docs
update_docs = docs.copy()
for c in corrections:
    for d in update_docs:        
        if d['name']==c['doc_id']:
            d['source'] = 'EMC_DCC_ORIGINAL_ADJUSTED'
            for a in d['annotations']:
                if a['id']==c['annotation_id']:
                    a['meta_anns'][c['meta']]['value'] = c['value']
# put updated docs in DCC
DCC['projects'][0]['documents'] = update_docs

In [ ]:
# write DCC back to json 
with open('../data/emc-dcc_ann_ADJ.json', 'w') as f:
    json.dump(DCC, f, indent=2)

In [ ]:
relevant_docs_hypothetical = []
for i, doc in enumerate(docs):
    for concept in doc['annotations']:
        if (concept['meta_anns']['Temporality']['value']=='hypothetical'):
            doc['index'] = i
            relevant_docs_hypothetical.append(doc)
            break
        
relevant_docs_experiencer = []
for i, doc in enumerate(docs):
    for concept in doc['annotations']:
        if (concept['meta_anns']['Experiencer']['value']=='other'):
            doc['index'] = i
            relevant_docs_experiencer.append(doc)
            break

In [ ]:
OAI_ASYNC_CLIENT = AsyncOpenAI(api_key=os.getenv("OPENAI_KEY"), max_retries=2)
OAI_CLIENT = OpenAI(api_key=os.getenv("OPENAI_KEY"), max_retries=2)

In [ ]:
SYSTEM_PROMPT_HYPOTHETICAL = """
    Je bent een kritische assistent die mij helpt om nieuwe tekst te bedenken.
    De tekst moeten voldoen aan de volgende eisen:
    - ze moeten semantisch correct zijn en vergelijkbaar zijn met de voorbeeltekst die ik je geef.
    - de voorbeeltekst wordt voorafgegaan door de term VOORBEELDTEKST
    - in de voorbeeldzin worden 1 of meer concepten benoemd die hypothethisch zijn, het is belangrijk
    dat deze concepten in de nieuwe zin ook hypothetisch zijn, het mogen ook andere concepten zijn. 
    Een voorbeeld van een hypothetische concept = 'een voorafgaand trauma kan niet worden herinnerd', waarin 'trauma' het concept is.
    Een ander voorbeeld = 'ter uitsluiting van epifysaire dysplasie' waarin 'epifysaire dysplasie' het concept is.
    - de concepten die je moet vervangen zijn aangegeven met verticale streepjes, dus |concept|.
    - het domein is medisch dus gebruik medische concepten.
    - probeer de medische concepten te varieren, dus gebruik niet steeds dezelfde concepten.
    - geef als antwoord ALLEEN de nieuw gegenereerde zinnen, voorafgaand met de term NIEUWE_TEKST
    - in de NIEUWE_TEKST, plaats de concepten die hypothetisch zijn tussen verticale streepjes, dus '|', 
    dus bijvoorbeeld: 'ter uitsluiting van |epifysaire dysplasie|'
    
In case you have doubts:
'Hypothetical' is specifically about (theoretical) concepts, which means concepts that are not (yet) realized OR
concepts that may have occurred in the past. 'Historical' and 'Recent' can be used for realized concepts, in which we also include their negations. 
I.e. if a concept is explicitly denied historically or recently, we can label it as 'Historical' or 'Recent' respectively.
"""


SYSTEM_PROMPT_EXPERIENCER = """
    Je bent een kritische assistent die mij helpt om nieuwe text te bedenken.
    Deze text moeten voldoen aan de volgende eisen:
    - het moet semantisch correct zijn en vergelijkbaar zijn met de text die ik je geef.
    - de voorbeeldtext wordt voorafgegaan door de term VOORBEELDTEKST
    - in de voorbeeldtext worden 1 of meer concepten benoemd die verwijzen naar een persoon anders dan de patient, het is belangrijk
    dat deze concepten in de nieuwe zin ook verwijzen naar iemand anders dan de patient (zoals een familielid), 
    het mogen ook andere medische concepten zijn.
    - de concepten die je moet vervangen zijn aangegeven met verticale streepjes, dus |concept|.
    - Een voorbeeld van een concept wat verwijst naar een ander persoon dan de patient =
    'Een zusje van #Name# is elders operatief behandeld in verband met recidiverende patella luxaties', waarin 'luxaties' het concept is, en er 
    wordt verwezen naar de zus van de patient.    
    - het domein is medisch dus gebruik medische concepten.
    - probeer de medische concepten te varieren, dus gebruik niet steeds dezelfde concepten.
    - varieer de ziektebeelden
    - varieer de opmaak van de text, dus gebruik niet steeds dezelfde opmaak.
    - geef als antwoord ALLEEN de nieuw gegenereerde text, voorafgaand met de term NIEUWE_TEKST
    - in de NIEUWE_TEKST, plaats alleen de concepten die verwijzen naar een ander persoon dan de patient tussen tussen verticale streepjes |, 
    dus bijvoorbeeld: 'Een zusje van #Name# is elders operatief behandeld in verband met recidiverende patella |luxaties|'
"""

In [ ]:
def get_chat_res(USER_TEXT='Good day', 
                 SYSTEM_PROMPT=SYSTEM_PROMPT_HYPOTHETICAL, 
                 n = 10,
                 MODEL="gpt-4"):
    return OAI_CLIENT.chat.completions.create(
            model=MODEL,
            n = n,
            messages=[
                        {"role": "system",
                        "content": SYSTEM_PROMPT
                        },
                        {"role": "user", 
                        "content": USER_TEXT
                        }],
            stream=False,
        )

In [ ]:
GPT_VERSION = 'gpt-4-1106-preview'
CURRENT_DATE = datetime.datetime.now().strftime("%Y%m%d")

In [ ]:
#re_extract = re.compile(r'NIEUWE_ZIN\:(.*)')
if run_temporality:
    nieuwe_zinnen_hypothetisch = []
    for i, doc in tqdm(enumerate(relevant_docs_hypothetical)):
        EXAMPLE = doc['text'].replace('|', ' ')
        # add | vertical bars around the concept that needs to be replaced
        LOCS = [(d['start'],d['end']) for d in doc['annotations'] 
                    if d['meta_anns']['Temporality']['value']=='hypothetical']
        for loc in LOCS:
            EXAMPLE = EXAMPLE[:loc[0]] + '|' + EXAMPLE[loc[0]:loc[1]] + '|' + EXAMPLE[loc[1]:]
        
        res = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT_HYPOTHETICAL, 
                        n=10,
                        MODEL=GPT_VERSION, # gpt-3.5-turbo-instruct-0914
                        USER_TEXT="VOORBEELDTEKST: " + EXAMPLE)

        for j, _res in enumerate(res.choices):
            txt = _res.message.content
            nieuwe_zinnen_hypothetisch.append((
                doc['name'],
                'hypothetical',
                j,
                txt[txt.find('NIEUWE_TEKST')+12:].strip()))    
    
    hypothetical_df = pd.DataFrame(nieuwe_zinnen_hypothetisch, columns=['doc_id', 'class_value', 'synth_num', 'text'])
    hypothetical_df.to_parquet(f'../data/synth_temporality_{GPT_VERSION.replace("-", "_")}_{CURRENT_DATE}.parquet')
else:
    hypothetical_df = pd.read_parquet(hypothetical_file)
    hypothetical_df['class_value'] = 'hypothetical'

In [ ]:
#re_extract = re.compile(r'NIEUWE_ZIN\:(.*)')
if run_experiencer:
    nieuwe_zinnen_experiencer = []
    for i, doc in tqdm(enumerate(relevant_docs_experiencer)):
        EXAMPLE = doc['text'].replace('|', ' ')
        # add | vertical bars around the concept that needs to be replaced
        LOCS = [(d['start'],d['end']) for d in doc['annotations'] 
                    if d['meta_anns']['Experiencer']['value']=='other']
        for loc in LOCS:
            EXAMPLE = EXAMPLE[:loc[0]] + '|' + EXAMPLE[loc[0]:loc[1]] + '|' + EXAMPLE[loc[1]:]
        
        res = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT_EXPERIENCER, 
                        n=10,
                        MODEL=GPT_VERSION, # gpt-3.5-turbo-instruct-0914
                        USER_TEXT="VOORBEELDTEKST: " + EXAMPLE)

        for j, _res in enumerate(res.choices):
            txt = _res.message.content
            nieuwe_zinnen_experiencer.append((
                doc['name'],
                'other',
                j,
                txt[txt.find('NIEUWE_TEKST')+12:].strip()))
    experiencer_df = pd.DataFrame(nieuwe_zinnen_experiencer, columns=['doc_id', 'class_value', 'synth_num', 'text'])
    experiencer_df.to_parquet(f'../data/synth_experiencer_{GPT_VERSION.replace("-", "_")}.parquet')
else:
    experiencer_df = pd.read_parquet(experiencer_file)
    experiencer_df['class_value'] = 'other'


### Extract the spans from the synthetic set and add them to the original dataset with an additional label

In [99]:
def put_in_dict(data, original_documents, Class_name='Experiencer', Class_value=None):
    new_documents = original_documents.copy()
    
    if type(data)==pd.DataFrame:
        data = [(r.doc_id, r.class_value, r.synth_num, r.text)  for r in data.itertuples()]
    
    for i, (name, Class_value_spec, subid, text) in enumerate(data):
        Class_value_spec = Class_value_spec if Class_value_spec is not None else Class_value
        
        _doc = {
            'id': i,
            'source': 'synthetic',
            'source_version': f"{GPT_VERSION}|{CURRENT_DATE}",     
            'name': f"{name}|synth|{subid}",
            'text': text.replace('|', ''),
            'annotations' : []
        }
        
        try:
            for match in re.finditer('\|[A-zÀ-ÿ\s]+\|', text):
                start, end = match.span()
                start += 1
                end -= 1
                _doc['annotations'].append(
                            {
                                'id': 1,
                                'user': 'emc_dcc_synth',
                                'cui': 1,
                                'start': start,
                                'end': end,
                                'value': text[start:end],
                                'validated': False,
                                'correct': True,
                                'alternative': False,
                                'killed': False,
                                'meta_anns': {
                                    Class_name: {'value': Class_value_spec,
                                                    'name': Class_name,
                                                    'validated': False,
                                                    'acc': 1.0
                                                    },
                                }            
                            }
                        )         
        except:
            print(i, text)
        new_documents.append(_doc)
    return new_documents

In [100]:
experiencer_df = experiencer_df[['doc_id', 'class_value', 'synth_num', 'text']]
hypothetical_df = hypothetical_df[['doc_id', 'class_value', 'synth_num', 'text']]

In [101]:
new_docs_experiencer = put_in_dict(experiencer_df, update_docs, Class_name='Experiencer', Class_value='other')
new_docs_temporality = put_in_dict(hypothetical_df, new_docs_experiencer, Class_name='Temporality', Class_value='hypothetical')

In [104]:
new_docs_temporality[-1]

{'id': 689,
 'source': 'synthetic',
 'source_version': 'gpt-4-1106-preview|20231218',
 'name': 'SP2114|synth|9',
 'text': ': Geen symptomen die duiden op colitis ulcerosa of intestinale metaplasie.\nGeen tekenen van colitis ulcerosa waargenomen. Advies: Bij aanhoudende onzekerheid over de intestinale metaplasie een elektieve gastroscopie overwegen na overleg.',
 'annotations': [{'id': 1,
   'user': 'emc_dcc_synth',
   'cui': 1,
   'start': 32,
   'end': 48,
   'value': 'colitis ulcerosa',
   'validated': False,
   'correct': True,
   'alternative': False,
   'killed': False,
   'meta_anns': {'Temporality': {'value': 'hypothetical',
     'name': 'Temporality',
     'validated': False,
     'acc': 1.0}}},
  {'id': 1,
   'user': 'emc_dcc_synth',
   'cui': 1,
   'start': 54,
   'end': 76,
   'value': 'intestinale metaplasie',
   'validated': False,
   'correct': True,
   'alternative': False,
   'killed': False,
   'meta_anns': {'Temporality': {'value': 'hypothetical',
     'name': 'Tempor

## Write new samples to the dataset

In [105]:
DCC['projects'][0]['documents'] = new_docs_temporality

In [106]:
# write DCC back to json 
with open('../data/emc-dcc_ann_Augmented.json', 'w') as f:
    json.dump(DCC, f, indent=2)

## Translate English corpora

* English -- BioScope; [HF](https://huggingface.co/datasets/bigbio/bioscope), [src](https://rgai.inf.u-szeged.hu/downloads)
* English -- [Genia](http://www.geniaproject.org/genia-corpus/term-corpus)
* English -- Sherlock, SFU review corpus
